In this notebook we will generate and save the data frames that we will use in visualization, using the clean ones done in the previous steps

In [10]:
import pandas as pd
sys.path.append('../')
import src.limpieza as lmp

In [7]:
df_civitatis_clean = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\civitatis_clean.pkl')
df_turistas_admon = pd.read_pickle(r'C:\Users\mituc\Ironhack\Curso\IronLabs\Proyecto-1\data\Barcelona_guiris_esquilados.pkl')

# Civitatis

Total number of tourist

In [8]:
df_civitatis_clean.head(5)

,fecha,nombre,procedencia,opinion,viajo_con
0,2022-03-11,Ana,"Madrid, España","Victor es encantador, te explica todo al detal...",Viajó con hijos mayores
1,2022-03-11,Montserrat,España,"Casi 3 horas de tour pero Víctor, el guía que ...",Viajó solo
2,2022-03-11,Ainhoa,España,,Viajó en pareja
3,2022-03-11,Patricia Suárez Carrillo,"Madrid, España",A mi pareja y a mi nos ha encantado! Nuestro g...,Viajó en pareja
4,2022-03-11,Anónimo,,,


In [ ]:
lmp.total_civitatis_tourists_month(df_civitatis_clean)